# 千帆function_call工具调用

## 简介

上一节[千帆function_call入门](./agents/langchain_agent_with_qianfan_llm.ipynb)展示了实现chat调用函数的功能，本节将介绍如何让chat与千帆工具进行交互，并编写更便利的调用函数。

## 准备

本文使用的模块如下：
- 千帆 Python SDK中的 chat_completion 模块，该模块提供了与千帆对话引擎的交互接口；
- 千帆 tool中的duckduckgo_search_tool，该工具提供了网络搜索功能。

首先安装千帆 Python SDK，版本号 >= 0.2.6。

In [252]:
%%capture
!pip install "qianfan>=0.2.6" -U

初始化我们所需要的凭证

In [253]:
# 初始化LLM
import os

# qianfan sdk 鉴权
os.environ["QIANFAN_AK"] = "your_ak"
os.environ["QIANFAN_SK"] = "your_sk"

自定义函数库，此处调用qianfan.common.tool中的duckduckgo搜索引擎工具进行搜索，编写today函数查询今天日期。
同时，编写warp_json装饰器，将函数返回值转化为json格式；编写word_count函数防止返回值超过字数限制。

In [254]:
from qianfan.common.tool import duckduckgo_search_tool
from datetime import date
import json

search_engine = None
def warp_json(func):
    def wrapper(*args, **kwargs):
        return json.dumps({
            "return":func(*args, **kwargs)
        },
            ensure_ascii=False
        )
    return wrapper

def word_count(text: dict[str,str]) -> int:
    count=0
    for key, value in text.items():
        count += len(value)
    return count

@warp_json
def search(search_query: str) -> str:
    global search_engine
    if search_engine is None:
        search_engine = duckduckgo_search_tool.DuckDuckGoSearchTool()
    search_result = search_engine.run({"search_query":search_query})
    return_result = []
    count = 0
    for i, result_count in enumerate(map(word_count, search_result)):
        if count + result_count > 3000:
            break
        count += result_count
        return_result.append(search_result[i])
    # return json.dumps({"return":return_result}, ensure_ascii=False)
    return return_result

@warp_json
def today()->str:
    current_date = date.today().strftime("%Y-%m-%d")
    # return json.dumps({"return":current_date})
    return current_date
    

描述函数调用策略
其中，name为函数名称，description为函数描述，properties为入参schema，required为必填参数
properties支持json schema格式，具体参考[json schema](https://json-schema.org/learn/getting-started-step-by-step.html


In [255]:
funcs = [{
    "name": "search",  # 函数名称
    "description": "在网络上搜索信息，返回一个列表，列表元素为字典，字典有两个字段，分别为title：标题，content：搜索内容，有时会有第三个字段url：搜索指向的网页url。如果有url字段，请以超链接返回",  # 函数描述
    "parameters":{
        "type":"object",
        "properties":{  # 参数schema，如果参数为空，为空字典即可
            "search_query":{  # 参数名称
                "type":"string",  # 参数类型
                "description": "搜索内容"  # 参数描述
            }
        },
        "required":["search_query"]  # 必填参数（无默认值）
    }
},{
    "name": "today",
    "description": "获取今天的日期，格式为YYYY-MM-DD",
    "parameters":{
        "type":"object",
        "properties":{},
        "required":[]
    }
}]

定义函数名映射，方便function_call调用

In [256]:
func_name_map = {
    "search":search,
    "today":today
}

将对话过程封装成函数，方便多轮对话调用

In [257]:
import qianfan

def chat(chat_completion, messages, functions, function_map):
    resp = chat_completion.do(
        messages=messages,
        functions = functions
    )
    
    if resp.get("function_call"):
        print("**call function**")
        
        func_call = resp["function_call"]
        func_name = func_call["name"]
        arguments = json.loads(func_call["arguments"])
        func_content = function_map[func_name](**arguments)
        
        messages.append(resp, role="assistant")
        messages.append(func_content, role="function")
        
        second_resp = chat_completion.do(
            messages=messages,
            functions=functions
        )
        resp_content = second_resp["result"]
    else:
        print("**No function**")
        resp_content = resp["result"]
    print(resp_content)
    msgs.append(resp_content, role="assistant")

开始对话，首先获取今天的日期。

In [259]:
chat_comp = qianfan.ChatCompletion(model="ERNIE-Bot-4")
query = "今天几号？"
msgs = qianfan.Messages()
msgs.append(query,role="user")
chat(chat_comp, msgs,funcs, func_name_map)

**call function**
今天是2024年1月12日。如果您还有其他问题或需要更多信息，请随时告诉我。


接下来进行多轮对话，用日期信息来查询新闻。

In [260]:
query = "那么根据刚才的日期，今天有什么新闻么？请列举出标题、摘要并标注日期"
msgs.append(query,role="user")
chat(chat_comp, msgs,funcs, func_name_map)

**call function**
根据您的要求，以下是2024年1月12日的部分新闻列表：

1. 标题：China's consumer prices stuck in decline, factory-gate deflation ...
摘要：1月12日，路透社报道，中国的消费者价格在12月份连续第三个月下降，而工厂出厂价格也下降，凸显了经济放缓的压力。
2. 标题：从数字看发展 | 新一轮财税改革目标重点在哪？
摘要：第一财经报道，中央提出谋划新一轮财税体制改革后，财税改革再次受到关注。本文将从数字角度解读新一轮财税改革的目标和重点。
3. 标题：春运首日火车票今日开抢 规则有变!
摘要：中国新闻网报道，2024年春运自1月26日开始至3月5日结束，共计40天。春运首日（1月26日）的火车票将于1月12日开售。今年的候补购票功能、车票改签范围均有所调整。同时，铁路12306手机客户端推出了购票需求预填和火车票起售提醒订阅两项新功能，旅客购票将更加方便快捷。

以上是部分新闻的标题、摘要和日期。如果您需要查看更多新闻，请随时告诉我。


In [262]:
query = "还有哪些新闻？"
msgs.append(query,role="user")
chat(chat_comp, msgs,funcs, func_name_map)

**call function**
除了之前提到的新闻，还有一些其他国际新闻，包括“欧洲整合之父”、欧盟委员会前主席雅克·德洛尔去世、美国之音中文网的即时新闻报道以及纽约时报中文网的国际纵览等。这些新闻涵盖了国际政治、文化、娱乐等多个方面。如果您需要查看更多新闻，请随时告诉我。


In [264]:
query = "总结一下，这些新闻涉及了哪些主题？"
msgs.append(query,role="user")
chat(chat_comp, msgs,funcs, func_name_map)

**call function**
根据您提供的新闻，这些新闻涉及了多个主题，包括国内、国际、经济、社会、法治、文娱、科技、生活、军事等各个领域。具体来说，这些新闻涵盖了政治、经济、文化、娱乐、科技等多个方面，其中涉及的主题包括中国经济、财税改革、春运、外交关系、天气、科技等。此外，还有一些新闻涉及国际政治、文化、娱乐等多个方面，包括“欧洲整合之父”、欧盟委员会前主席雅克·德洛尔去世、美国之音中文网的即时新闻报道以及纽约时报中文网的国际纵览等。这些新闻主题多样，内容丰富，可以让读者了解到各个领域的最新动态和趋势。
